# Example notebook 01

Using the data generated from notebook `00_create_data.ipynb`, this notebook takes you through some of the basic functionality using the `DataCuration` class:
+ [Finding files](#Finding-files)
+ [Reading in data](#Reading-in-data)
+ [Convert to the correct dtypes](#Convert-to-the-correct-dtypes)
+ [\_\_repr_html\_\_](#__repr_html__)
+ [Set new header values](#Set-new-header-values)
+ [Disable the logging messages](#Disable-the-logging-messages)
+ [Set the data](#Set-the-data)
+ [Alter existing columns](#Alter-existing-columns)
+ [Assert nulls](#Assert-nulls)
+ [Forming summary tables](#Forming-summary-tables)

## Setup

<hr>

Imports and settings options

In [1]:
import IPython.core.display as ICD
import pickle
import os
from datetime import datetime
import glob
import logging

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 12)
pd.set_option('display.float_format', '{:,.2f}'.format)

logger = logging.getLogger()

In [2]:
from data_etl import DataCuration

## Examples
<hr>

### Finding files
<hr>

Initialise the class

In [3]:
var_start_time = datetime.now()
dc_data = DataCuration(var_start_time, '1')

Define a function to search for the files and look at what files are found

In [4]:
def find_files(files_path):
    list_files = [os.path.abspath(file) for file in glob.glob(f'{files_path}/*_convert*.tsv')]
    return list_files

Apply it to the `dc_data` object to get a list of the files

In [5]:
dc_data.find_files(function=lambda files_path: find_files(files_path), files_path='data')
dc_data.list_files

['C:\\Users\\georg\\Documents\\workspace\\modules\\data_etl\\examples\\data\\df_convert.tsv',
 'C:\\Users\\georg\\Documents\\workspace\\modules\\data_etl\\examples\\data\\df_convert_issues.tsv']

### Reading in data
<hr>

Continuing on from the `dc_data` object defined above, define a function to read in the files

In [6]:
def read_in(list_files):
    dict_tables = dict()
    for file in list_files:
        var_filename = file.split('\\')[-1]
        dict_tables[var_filename] = pd.read_csv(file, sep='\t', dtype=str)
    return dict_tables

Apply it to the `dc_data` object to get the data

In [7]:
dc_data.reading_in(function=read_in)
dc_data.tables

{'df_convert.tsv':   object int float        date
 0      A   1   0.6  2019-01-01
 1      B   4   5.2  2019-02-05
 2      C   1   5.6  2018-12-17
 3      D  10  15.9  2019-07-18
 4      E  -8   4.7  2018-03-09,
 'df_convert_issues.tsv':   object  int float        date
 0      A    1   0.6  2019-02-29
 1      B  4.5     A  2019-22-05
 2      C    1   5.6  2018-12-17
 3      D    b  15.9  2019-09-31
 4      5   -8   4.7  2018-03-09}

### Convert to the correct dtypes
<hr>

Continuing on from the `dc_data` object defined above, define the functions for converting columns to specific dtypes

In [8]:
def func_try_float_cnv(x): 
    try:
        var = float(x)
    except:
        return True
    return False


def func_try_int_cnv(x): 
    try:
        var = int(x)
    except:
        return True
    return False


def func_str_cnv(s):
    var_is_null_pre = s.isnull().sum()
    s_cnv = s.map(func_to_int).str.strip()
    var_is_null_post = s_cnv.isnull().sum()
    if var_is_null_post != var_is_null_pre:
        raise ValueError
    return s_cnv


def func_to_int(x):
    try:
        return int(x)
    except: 
        return x


def func_try_str_cnv(s):
    var_is_null_pre = s.isnull().sum()
    s_cnv = s.map(func_to_int).str.strip()
    var_is_null_post = s_cnv.isnull().sum()
    return s != s_cnv


def func_try_date_cnv(x):
    if pd.isnull(x):
        return False
    if pd.isnull(pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')):
        return True
    return False

Define the dictionary used by the `DataCuration` class that will do the conversions

In [9]:
dict_convert = {
    'float': {
        'columns': ['float'],
        'dtypes': ['float'],
        'functions': {
            1: lambda df, col, **kwargs: df[col].astype(float)
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_float_cnv)
    },
    'int': {
        'columns': ['int'],
        'dtypes': ['int'],
        'functions': {
            1: lambda df, col, **kwargs: df[col].astype(int)
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_int_cnv)
    },
    'object': {
        'columns': ['object'],
        'dtypes': [],
        'functions': {
            1: lambda df, col, **kwargs: func_str_cnv(df[col])
        },
        'idx_function': lambda df, col, **kwargs: func_try_str_cnv(df[col])
    },
    'date': {
        'columns': ['date'],
        'dtypes': ['date', '[ns]'],
        'functions': {
            1: lambda df, col, **kwargs: pd.to_datetime(df[col], format='%Y-%m-%d')
        },
        'idx_function': lambda df, col, **kwargs: df[col].map(func_try_date_cnv)
    }
}

Apply it to the `dc_data` object to convert the data, the warnings are for where problems were found, these are the logging messages the class outputs

In [10]:
dc_data.convert_columns(dictionary=dict_convert)

The conversion failed for keys float, 1, trying next
The conversion for column float for convert_key float failed.
The conversion failed for keys int, 1, trying next
The conversion for column int for convert_key int failed.
The conversion failed for keys object, 1, trying next
The conversion for column object for convert_key object failed.
The conversion failed for keys date, 1, trying next
The conversion for column date for convert_key date failed.


A look at the table that has issues with the conversion

In [11]:
dc_data.tables['df_convert_issues.tsv']

,object,int,float,date
0,A,1,0.6,2019-02-29
1,B,4.5,A,2019-22-05
2,C,1,5.6,2018-12-17
3,D,b,15.9,2019-09-31
4,5,-8,4.7,2018-03-09


A look at the issues log that is created within the class, these link in with the warning messages a few cells above

In [12]:
dc_data.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format float,float,1,1,2020-05-25 20:34:18.277847
1,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format int,int,2,"1, 3",2020-05-25 20:34:18.277847
2,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format object,object,1,4,2020-05-25 20:34:18.277847
3,1,None,None,df_convert_issues.tsv,NaN,0,NaN,,The conversion failed to format date,date,3,"0, 1, 3",2020-05-25 20:34:18.277847


### \_\_repr_html\_\_
<hr>

A quick summary of some of the important information from the tables stored in the `dc_data` object, this output is configured for html representation

You can see that the `df_convert_issues.tsv` table has only got object columns

In [13]:
dc_data

### Set new header values
<hr>

As the tables have the same number of columns we can pass a list to assign the new headers

In [14]:
dc_data.set_headers(list_cols=['o_object', 'i_integer', 'f_float', 'd_datetime'])

Looking at the tables they both have the new header values

In [15]:
ICD.display(dc_data.tables['df_convert.tsv'])
ICD.display(dc_data.tables['df_convert_issues.tsv'])

,o_object,i_integer,f_float,d_datetime
0,A,1,0.60,2019-01-01
1,B,4,5.20,2019-02-05
2,C,1,5.60,2018-12-17
3,D,10,15.90,2019-07-18
4,E,-8,4.70,2018-03-09


,o_object,i_integer,f_float,d_datetime
0,A,1,0.6,2019-02-29
1,B,4.5,A,2019-22-05
2,C,1,5.6,2018-12-17
3,D,b,15.9,2019-09-31
4,5,-8,4.7,2018-03-09


You can also use a function input

In [16]:
dc_data.set_headers(function=lambda x: x.split('_')[1])

Looking at the tables they both have the transformed header values

In [17]:
ICD.display(dc_data.tables['df_convert.tsv'])
ICD.display(dc_data.tables['df_convert_issues.tsv'])

,object,integer,float,datetime
0,A,1,0.60,2019-01-01
1,B,4,5.20,2019-02-05
2,C,1,5.60,2018-12-17
3,D,10,15.90,2019-07-18
4,E,-8,4.70,2018-03-09


,object,integer,float,datetime
0,A,1,0.6,2019-02-29
1,B,4.5,A,2019-22-05
2,C,1,5.6,2018-12-17
3,D,b,15.9,2019-09-31
4,5,-8,4.7,2018-03-09


### Disable the logging messages
<hr>

Because the warning messages can get quick verbose in the notebook I will disable them, however in scripts the logging can be written out to a text file o be reviewed after or if there is a cause to debug

In [18]:
logger.setLevel(logging.CRITICAL)
# logger.setLevel(logging.INFO)  # To enable them logging messages again

### Set the data
<hr>

Instead of reading in you can also just set the data in the class

Initialise the class

In [19]:
var_start_time = datetime.now()
dc_alter = DataCuration(var_start_time, '2')

Set the tables in the `dc_alter` object as a dictionary of two tables

In [20]:
dc_alter.set_table(
    {
        'df_alterations.tsv': pd.DataFrame(
            [
                ('A', 2, 'key_1'),
                ('B', 199, 'key_2'),
                ('C', -1, 'key_1'),
                ('D', 20, 'key_3'),
                ('E', 6, 'key_2')
            ],
            columns=['to_map', 'add_1', 'merge_key']
        ),
        'df_alterations_issues.tsv': pd.DataFrame(
            [
                ('A', 2, 'key_1'),
                ('B', 199, 2),
                ('C', -1, 'key_1'),
                (['D'], 'a', 'key_3'),
                ('E', 6, 'key_2')
            ],
            columns=['to_map', 'add_1', 'merge_key']
        )
    }
)

Look at the tables

In [21]:
dc_alter.tables

{'df_alterations.tsv':   to_map  add_1 merge_key
 0      A      2     key_1
 1      B    199     key_2
 2      C     -1     key_1
 3      D     20     key_3
 4      E      6     key_2,
 'df_alterations_issues.tsv':   to_map add_1 merge_key
 0      A     2     key_1
 1      B   199         2
 2      C    -1     key_1
 3    [D]     a     key_3
 4      E     6     key_2}

### Alter existing columns
<hr>

Define the dictionary of information needed for altering the tables, this includes a DataFrame mapping and function to do something more complicated

In [22]:
df_mapping = pd.DataFrame(
    [
        ('key_1', 1),
        ('key_2', 2),
        ('key_3', 3)
    ], 
    columns=['merge_key', 'out_value']
)


def func_alter_merge(df, df_mapping):
    df_mapped = pd.merge(
        df,
        df_mapping,
        on='merge_key',
        how='left'
    )
    if (
        df_mapped['out_value'].isnull().sum() != 
        df['merge_key'].isnull().sum()
    ):
        raise ValueError
    return df_mapped


dict_alter = {
    '01': {
        'type': 'new_col',
        'col_name': 'key',
        'function': lambda df, keys, **kwargs: keys[0]
    },
    '02': {
        'type': 'new_col',
        'col_name': 'done_add_1',
        'function': lambda df, keys, **kwargs: df['add_1'] + 1,
        'idx_function': lambda df, keys, **kwargs: 
            df['add_1'].map(
                lambda x: type(x).__name__).map(
                lambda x: ('int' in x) | ('float' in x)).map(
                {True: False, False: True})
    },
    '03': {
        'type': 'new_col',
        'col_name': 'mapped',
        'function': lambda df, keys, **kwargs: df['to_map'].map({
            'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}),
        'idx_function': lambda df, keys, **kwargs: 
            ~df['to_map'].astype(str).isin(['A', 'B', 'C', 'D', 'E'])
    },
    '04': {
        'type': 'map_df',
        'function': lambda df, keys, **kwargs: 
            func_alter_merge(df, kwargs['df_mapping']),
        'idx_function': lambda df, keys, **kwargs: 
            ~df['merge_key'].isin(['key_1', 'key_2', 'key_3', np.nan])
    }
}

Apply it to the `dc_alter` object to alter the data

In [23]:
dc_alter.alter_tables(dictionary=dict_alter, df_mapping=df_mapping)

Look at the issues log from the alterations

In [24]:
dc_alter.df_issues

,key_1,key_2,key_3,file,sub_file,step_number,category,issue_short_desc,issue_long_desc,column,issue_count,issue_idx,grouping
0,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type new_col the function for alter_key 02...,done_add_1,1,3,2020-05-25 20:34:18.830713
1,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type new_col the function for alter_key 03...,mapped,1,3,2020-05-25 20:34:18.830713
2,2,None,None,df_alterations_issues.tsv,NaN,0,NaN,,For type map_df the function for alter_key 04 ...,NaN,1,1,2020-05-25 20:34:18.830713


And look at the tables stored in the object, we can see `df_alterations.tsv` didn't have any issues and now has more column while `df_alterations_issues.tsv` doesn't have all the new columns

In [25]:
ICD.display(dc_alter.tables['df_alterations.tsv'])
ICD.display(dc_alter.tables['df_alterations_issues.tsv'])

,to_map,add_1,merge_key,key,done_add_1,mapped,out_value
0,A,2,key_1,2,3,1,1
1,B,199,key_2,2,200,2,2
2,C,-1,key_1,2,0,3,1
3,D,20,key_3,2,21,4,3
4,E,6,key_2,2,7,5,2


,to_map,add_1,merge_key,key
0,A,2,key_1,2
1,B,199,2,2
2,C,-1,key_1,2
3,[D],a,key_3,2
4,E,6,key_2,2


### Assert nulls
<hr>

Instead of reading in you can also just set the data in the class

Initialise the class

In [26]:
var_start_time = datetime.now()
dc_nulls = DataCuration(var_start_time, '3')

Set the tables in the `dc_alter` object as a dictionary of two tables

In [27]:
dc_nulls.set_table(
    {
        'df_nulls.tsv': pd.DataFrame(
            [
                ('A', 'key_1', 'key_1'),
                ('B', 'key_2', 'key_2'),
                ('C', 'key_1', 'key_1'),
                ('D', 'key_3', 'key_3'),
                ('E', 'key_2', 'key_2')
            ],
            columns=['to_map', 'key', 'key_duplicate']
        )
    }
)

Use the `list_nulls` argument to pass a list of nulls you want to encode to null

In [28]:
dc_nulls.assert_nulls(list_nulls=['key_1'])

Look at the table

In [29]:
dc_nulls.tables['df_nulls.tsv']

,to_map,key,key_duplicate
0,A,NaN,NaN
1,B,key_2,key_2
2,C,NaN,NaN
3,D,key_3,key_3
4,E,key_2,key_2


Reset the tables in the object

In [30]:
dc_nulls.set_table(
    {
        'df_nulls.tsv': pd.DataFrame(
            [
                ('A', 'key_1', 'key_1'),
                ('B', 'key_2', 'key_2'),
                ('C', 'key_1', 'key_1'),
                ('D', 'key_3', 'key_3'),
                ('E', 'key_2', 'key_2')
            ],
            columns=['to_map', 'key', 'key_duplicate']
        )
    }
)

Use the `list_exclude_cols` argument to ignore certain columns so those column values are not considered

In [31]:
dc_nulls.assert_nulls(list_nulls=['key_1'], list_exclude_cols=['key'])

Look at the table

In [32]:
dc_nulls.tables['df_nulls.tsv']

,to_map,key,key_duplicate
0,A,key_1,NaN
1,B,key_2,key_2
2,C,key_1,NaN
3,D,key_3,key_3
4,E,key_2,key_2


### Forming summary tables
<hr>

Initialise the class

In [33]:
var_start_time = datetime.now()
dc_summary = DataCuration(var_start_time, '4')

Set the data

In [34]:
dc_summary.set_table(pickle.load(open('data/df_summary.pkl', 'rb')))

Define the functions required to create all the different summaries

In [35]:
def func_value_counts(df, col):
    s_out = df[col].value_counts()
    df_out = s_out.to_frame('count')
    df_out.reset_index(inplace=True)
    df_out.rename(columns={'index': col}, inplace=True)
    
    return df_out


def func_form_tables(tables, formed_tables, grouping, key_1, key_2, key_3, key_separator, **kwargs):
    dict_summaries = dict()
    if type(tables).__name__ == 'DataFrame':
        dict_summaries['original_table'] = tables
    dict_summaries['value counts on `str`'] = func_value_counts(tables, 'str')
    dict_summaries['value counts on `str_2`'] = func_value_counts(tables, 'str_2')
    
    return dict_summaries

Apply it to the `dc_summary` object to summarise the data

In [36]:
dc_summary.form_summary_tables(function=func_form_tables)

Have a look at the resulting summary tables

In [37]:
for key in dc_summary.formed_tables.keys():
    print(key)
    ICD.display(dc_summary.formed_tables[key])

original_table


,str,str_2,count,int_max
0,b,c,1,6
1,d,b,1,9
2,c,b,1,0
3,d,d,1,9
4,c,b,1,1
5,a,d,1,3
...,...,...,...,...
44,d,b,1,8
45,d,b,1,10
46,d,e,1,10


value counts on `str`


,str,count
0,a,17
1,d,14
2,c,10
3,b,9


value counts on `str_2`


,str_2,count
0,c,16
1,e,14
2,b,11
3,d,9


---
**GigiSR**